In [2]:
import psycopg2

In [3]:
try:
    # 1. DB 연결
    conn = psycopg2.connect(
        host="localhost", database="postgres", user="postgres", password="pg1234"
    )

    # 2. 커서 생성
    cur = conn.cursor()

    # 3. SQL 쿼리 실행
    print("PostgreSQL database version:")
    cur.execute("SELECT version();")

    # 4. 결과 가져오기
    db_version = cur.fetchone()
    print(db_version)

    # 5. 커서 및 연결 닫기
    cur.close()
    conn.close()

except Exception as e:
    print(f"Connection failed: {e}")

PostgreSQL database version:
('PostgreSQL 18.0 (Debian 18.0-1.pgdg13+3) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit',)


In [4]:
def get_connection():
    """
    DB 연결을 생성
    """
    return psycopg2.connect(
        host="localhost", database="postgres", user="postgres", password="pg1234"
    )


def setup_database():
    """
    데이터베이스 설정 함수
    """
    try:
        conn = get_connection()
        cur = conn.cursor()

        # 테이블 생성
        cur.execute("""
            CREATE TABLE IF NOT EXISTS users (
                id SERIAL PRIMARY KEY,
                name VARCHAR(100),
                email VARCHAR(100)
                );
        """)

        # 변경사항 커밋
        conn.commit()
        print("Database setup completed.")

        # 추가적인 데이터베이스 설정 작업 수행 가능
        cur.close()
    except Exception as e:
        print(f"Database setup failed: {e}")
    finally:
        if conn:
            conn.close()


def create_user(name, email):
    """
    Create new user in database
    """
    conn = None
    try:
        conn = get_connection()
        cur = conn.cursor()

        sql = "INSERT INTO users (name, email) VALUES (%s, %s) RETURNING id;"
        cur.execute(sql, (name, email))

        user_id = cur.fetchone()[0]

        conn.commit()
        print(f"User created successfully with ID: {user_id}")
        cur.close()

    except Exception as e:
        print(f"Failed to create user: {e}")
    finally:
        if conn:
            conn.close()


def get_user(user_id):
    """
    Retrieve user information by ID
    """
    conn = None
    try:
        conn = get_connection()
        cur = conn.cursor()

        sql = "SELECT id, name, email FROM users WHERE id = %s;"
        cur.execute(sql, (user_id,))
        user = cur.fetchone()
        cur.close()
        return user

    except Exception as e:
        print(f"Failed to retrieve user: {e}")
        return None
    finally:
        if conn:
            conn.close()


def get_all_users():
    """
    Read all users from db
    """
    conn = None
    try:
        conn = get_connection()
        cur = conn.cursor()

        cur.execute("SELECT id, name, email FROM users;")

        users = cur.fetchall()

        print("All users:")
        if not users:
            print("No users found.")
        else:
            for user in users:
                print(f" ID: {user[0]}, Name: {user[1]}, Email: {user[2]}")
        print("End of user list.")

        cur.close()
    except Exception as e:
        print(f"Failed to retrieve users: {e}")
    finally:
        if conn:
            conn.close()


def update_user_email(name, new_email):
    """
    Update user's email by name
    """
    conn = None
    try:
        conn = get_connection()
        cur = conn.cursor()

        sql = "UPDATE users SET email = %s WHERE name = %s;"
        cur.execute(sql, (new_email, name))
        conn.commit()

        print(f"Updated email for user '{name}' to '{new_email}'.")
        cur.close()
    except Exception as e:
        print(f"Failed to update user email: {e}")
    finally:
        if conn:
            conn.close()


def delete_user(name):
    """
    Delete user by name
    """
    conn = None
    try:
        conn = get_connection()
        cur = conn.cursor()

        sql = "DELETE FROM users WHERE name = %s;"
        cur.execute(sql, (name,))
        conn.commit()

        print(f"User '{name}' deleted successfully.")
        cur.close()
    except Exception as e:
        print(f"Failed to delete user: {e}")
    finally:
        if conn:
            conn.close()

In [5]:
setup_database()

create_user("Alice", "alice@example.com")
create_user("Bob", "bob@example.com")

read_user = get_user(1)
print(
    f"Retrieved User - ID: {read_user[0]}, Name: {read_user[1]}, Email: {read_user[2]}"
)

get_all_users()

update_user_email("Alice", "alice_new@example.com")

get_all_users()

delete_user("Bob")

get_all_users()

Database setup completed.
User created successfully with ID: 1
User created successfully with ID: 2
Retrieved User - ID: 1, Name: Alice, Email: alice@example.com
All users:
 ID: 1, Name: Alice, Email: alice@example.com
 ID: 2, Name: Bob, Email: bob@example.com
End of user list.
Updated email for user 'Alice' to 'alice_new@example.com'.
All users:
 ID: 2, Name: Bob, Email: bob@example.com
 ID: 1, Name: Alice, Email: alice_new@example.com
End of user list.
User 'Bob' deleted successfully.
All users:
 ID: 1, Name: Alice, Email: alice_new@example.com
End of user list.
